## TabNet
TabNetRegressor : simple and multi-task regression problems
- https://github.com/dreamquark-ai/tabnet/blob/develop/multi_regression_example.ipynb
- https://dreamquark-ai.github.io/tabnet/generated_docs/README.html#model-parameters
- https://arxiv.org/pdf/1908.07442.pdf

In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

warnings.filterwarnings("ignore")

## p10

In [3]:
train = pd.read_csv("p10_train.csv")
train['istrain'] = 1
test = pd.read_csv("p10_test.csv")
test['istrain'] = 0
df = pd.concat([train, test]).reset_index(drop=True)

X_name = [f'element {i}' for i in range(1,13)]
Y_name = [f'property {i}' for i in range(1,7)]

poolx = df[X_name].copy(deep=True)
pooly = df[Y_name].copy(deep=True)

X = poolx.iloc[df[df['istrain']==1].index, :]
Y = pooly.iloc[df[df['istrain']==1].index, :]

X_test = poolx.iloc[df[df['istrain']==0].index, :].reset_index(drop=True)
Y_test = pooly.iloc[df[df['istrain']==0].index, :].reset_index(drop=True)

pred = torch.zeros((X_test.shape[0], 6))

X.shape

(1372, 12)

In [5]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error



use_fea = X.columns.tolist()
cat_idxs = []
cat_dims = []# qcut bins
cat_emb_dim = []
max_epochs = 1000

oof_train =  np.zeros(Y.shape)
oof_real_train = torch.zeros((X.shape[0], 6))
models = []

kf = KFold(n_splits=5, random_state=2022, shuffle=True)
for now_fold, (trn_idx, val_idx) in enumerate(kf.split(X, Y)):
    print(f"Fold: {now_fold+1}")
    X_train, X_val = X.iloc[trn_idx].values, X.iloc[val_idx].values
    y_train, y_val = Y.iloc[trn_idx].values, Y.iloc[val_idx].values
    
    clf = TabNetRegressor(seed=2022)
    
    clf.fit(
        X_train=X_train, y_train=y_train, 
        eval_set = [(X_train, y_train), (X_val, y_val)],
        eval_name=['train', 'valid'],
        eval_metric=['mae', 'rmse', 'mse', ], 
        max_epochs=max_epochs, 
        patience=50,
        batch_size=128, virtual_batch_size=32,
        num_workers=0,
        drop_last=False)
    

    oof_train[val_idx] = clf.predict(X_val)
    oof_real_train[trn_idx] += clf.predict(X_train) / 4
    
    pred += clf.predict(X_test.values) / 5
    
    models.append(clf)
    valid_mse = mean_squared_error(y_pred=oof_train[val_idx], y_true=y_val)


    print(f"FINAL VALID SCORE : {valid_mse}")

Fold: 1
Device used : cuda
epoch 0  | loss: 1.55712 | train_mae: 0.63744 | train_rmse: 1.02277 | train_mse: 1.04605 | valid_mae: 0.69422 | valid_rmse: 1.93956 | valid_mse: 3.7619  |  0:00:00s
epoch 1  | loss: 0.90874 | train_mae: 0.58615 | train_rmse: 0.98096 | train_mse: 0.96229 | valid_mae: 0.63669 | valid_rmse: 1.91613 | valid_mse: 3.67155 |  0:00:00s
epoch 2  | loss: 0.78462 | train_mae: 0.54365 | train_rmse: 0.94885 | train_mse: 0.90031 | valid_mae: 0.58254 | valid_rmse: 1.89168 | valid_mse: 3.57845 |  0:00:01s
epoch 3  | loss: 0.66868 | train_mae: 0.47051 | train_rmse: 0.90382 | train_mse: 0.8169  | valid_mae: 0.50713 | valid_rmse: 1.86895 | valid_mse: 3.49298 |  0:00:01s
epoch 4  | loss: 0.62998 | train_mae: 0.42149 | train_rmse: 0.86491 | train_mse: 0.74806 | valid_mae: 0.45841 | valid_rmse: 1.85319 | valid_mse: 3.43431 |  0:00:02s
epoch 5  | loss: 0.60135 | train_mae: 0.41838 | train_rmse: 0.85594 | train_mse: 0.73263 | valid_mae: 0.45644 | valid_rmse: 1.84753 | valid_mse: 3.4

In [7]:
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, r2_score
print("train")

for i in range(Y.shape[1]):
    mae = mean_absolute_error(Y.iloc[:,i], oof_real_train[:,i])
    r2 = r2_score(Y.iloc[:,i], oof_real_train[:,i])
    rmse = np.sqrt(((Y.iloc[:, i] - np.array(oof_real_train[:,i])) ** 2).mean())
    pccs = pearsonr(Y.iloc[:, i], oof_real_train[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

print("valid")

for i in range(Y.shape[1]):
    mae = mean_absolute_error(Y.iloc[:,i], oof_train[:,i])
    r2 = r2_score(Y.iloc[:,i], oof_train[:,i])
    rmse = np.sqrt(((Y.iloc[:, i] - np.array(oof_train[:,i])) ** 2).mean())
    pccs = pearsonr(Y.iloc[:, i], oof_train[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

print("test")
for i in range(Y_test.shape[1]):
    mae = mean_absolute_error(Y_test.iloc[:,i], pred[:,i])
    r2 = r2_score(Y_test.iloc[:,i], pred[:,i])
    rmse = np.sqrt(((Y_test.iloc[:, i] - np.array(pred[:,i])) ** 2).mean())
    pccs = pearsonr(Y_test.iloc[:, i], pred[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

train
property1: mae:0.1285, rmse:0.6539, r2:0.8636, pccs:0.9581
property2: mae:0.0669, rmse:0.1015, r2:0.9868, pccs:0.9946
property3: mae:0.1332, rmse:0.3308, r2:0.8318, pccs:0.9156
property4: mae:0.079, rmse:0.1059, r2:0.9874, pccs:0.9955
property5: mae:0.1962, rmse:1.5958, r2:0.0479, pccs:0.2197
property6: mae:0.5335, rmse:0.7103, r2:0.4617, pccs:0.691
valid
property1: mae:0.1727, rmse:0.7027, r2:0.8424, pccs:0.9327
property2: mae:0.0967, rmse:0.1519, r2:0.9704, pccs:0.9859
property3: mae:0.1583, rmse:0.3548, r2:0.8065, pccs:0.9004
property4: mae:0.1083, rmse:0.1613, r2:0.9708, pccs:0.9867
property5: mae:0.2229, rmse:1.6168, r2:0.0226, pccs:0.1562
property6: mae:0.5639, rmse:0.7428, r2:0.4113, pccs:0.6432
test
property1: mae:0.1026, rmse:0.2363, r2:0.5214, pccs:0.7693
property2: mae:0.1885, rmse:0.2128, r2:0.9459, pccs:0.9961
property3: mae:0.2181, rmse:0.3316, r2:0.8064, pccs:0.9185
property4: mae:0.2002, rmse:0.2245, r2:0.9453, pccs:0.9975
property5: mae:0.1093, rmse:0.5356, r2:0.

## p11

In [8]:
train = pd.read_csv("p11_train_scaling_x.csv")
train['istrain'] = 1
test = pd.read_csv("p11_test_scaling_x.csv")
test['istrain'] = 0
df = pd.concat([train, test]).reset_index(drop=True)

X_name = [f'element {i}' for i in range(1,13)]
Y_name = [f'property {i}' for i in range(1,7)]

poolx = df[X_name].copy(deep=True)
pooly = df[Y_name].copy(deep=True)

X = poolx.iloc[df[df['istrain']==1].index, :]
Y = pooly.iloc[df[df['istrain']==1].index, :]

X_test = poolx.iloc[df[df['istrain']==0].index, :].reset_index(drop=True)
Y_test = pooly.iloc[df[df['istrain']==0].index, :].reset_index(drop=True)

pred = torch.zeros((X_test.shape[0], 6))

X.shape

(1372, 12)

In [9]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error



use_fea = X.columns.tolist()
cat_idxs = []
cat_dims = []# qcut bins
cat_emb_dim = []
max_epochs = 1000

oof_train =  np.zeros(Y.shape)
oof_real_train = torch.zeros((X.shape[0], 6))
models = []

kf = KFold(n_splits=5, random_state=2022, shuffle=True)
for now_fold, (trn_idx, val_idx) in enumerate(kf.split(X, Y)):
    print(f"Fold: {now_fold+1}")
    X_train, X_val = X.iloc[trn_idx].values, X.iloc[val_idx].values
    y_train, y_val = Y.iloc[trn_idx].values, Y.iloc[val_idx].values
    
    clf = TabNetRegressor(seed=2022)
    
    clf.fit(
        X_train=X_train, y_train=y_train, 
        eval_set = [(X_train, y_train), (X_val, y_val)],
        eval_name=['train', 'valid'],
        eval_metric=['mae', 'rmse', 'mse', ], 
        max_epochs=max_epochs, 
        patience=50,
        batch_size=128, virtual_batch_size=32,
        num_workers=0,
        drop_last=False)
    

    oof_train[val_idx] = clf.predict(X_val)
    oof_real_train[trn_idx] += clf.predict(X_train) / 4
    
    pred += clf.predict(X_test.values) / 5
    
    models.append(clf)
    valid_mse = mean_squared_error(y_pred=oof_train[val_idx], y_true=y_val)


    print(f"FINAL VALID SCORE : {valid_mse}")

Fold: 1
Device used : cuda
epoch 0  | loss: 1.42193 | train_mae: 0.63846 | train_rmse: 1.02164 | train_mse: 1.04375 | valid_mae: 0.6854  | valid_rmse: 1.93812 | valid_mse: 3.7563  |  0:00:00s
epoch 1  | loss: 0.943   | train_mae: 0.53902 | train_rmse: 0.91939 | train_mse: 0.84529 | valid_mae: 0.59961 | valid_rmse: 1.89289 | valid_mse: 3.58303 |  0:00:00s
epoch 2  | loss: 0.80265 | train_mae: 0.44347 | train_rmse: 0.85547 | train_mse: 0.73183 | valid_mae: 0.50467 | valid_rmse: 1.85518 | valid_mse: 3.44168 |  0:00:01s
epoch 3  | loss: 0.67797 | train_mae: 0.36653 | train_rmse: 0.80153 | train_mse: 0.64245 | valid_mae: 0.43003 | valid_rmse: 1.83079 | valid_mse: 3.3518  |  0:00:01s
epoch 4  | loss: 0.61558 | train_mae: 0.30017 | train_rmse: 0.76002 | train_mse: 0.57763 | valid_mae: 0.35505 | valid_rmse: 1.81066 | valid_mse: 3.27849 |  0:00:01s
epoch 5  | loss: 0.58634 | train_mae: 0.29536 | train_rmse: 0.75199 | train_mse: 0.56549 | valid_mae: 0.35956 | valid_rmse: 1.80781 | valid_mse: 3.2

In [10]:
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, r2_score
print("train")

for i in range(Y.shape[1]):
    mae = mean_absolute_error(Y.iloc[:,i], oof_real_train[:,i])
    r2 = r2_score(Y.iloc[:,i], oof_real_train[:,i])
    rmse = np.sqrt(((Y.iloc[:, i] - np.array(oof_real_train[:,i])) ** 2).mean())
    pccs = pearsonr(Y.iloc[:, i], oof_real_train[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

print("valid")

for i in range(Y.shape[1]):
    mae = mean_absolute_error(Y.iloc[:,i], oof_train[:,i])
    r2 = r2_score(Y.iloc[:,i], oof_train[:,i])
    rmse = np.sqrt(((Y.iloc[:, i] - np.array(oof_train[:,i])) ** 2).mean())
    pccs = pearsonr(Y.iloc[:, i], oof_train[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

print("test")
for i in range(Y_test.shape[1]):
    mae = mean_absolute_error(Y_test.iloc[:,i], pred[:,i])
    r2 = r2_score(Y_test.iloc[:,i], pred[:,i])
    rmse = np.sqrt(((Y_test.iloc[:, i] - np.array(pred[:,i])) ** 2).mean())
    pccs = pearsonr(Y_test.iloc[:, i], pred[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

train
property1: mae:0.1638, rmse:1.7134, r2:0.0634, pccs:0.9769
property2: mae:0.0763, rmse:0.1268, r2:0.9794, pccs:0.99
property3: mae:0.136, rmse:0.3274, r2:0.8353, pccs:0.9158
property4: mae:0.0768, rmse:0.1114, r2:0.986, pccs:0.9934
property5: mae:0.216, rmse:1.6163, r2:0.0233, pccs:0.2335
property6: mae:0.5187, rmse:0.6804, r2:0.506, pccs:0.7149
valid
property1: mae:0.1476, rmse:0.4484, r2:0.9358, pccs:0.9678
property2: mae:0.1008, rmse:0.1679, r2:0.9639, pccs:0.9819
property3: mae:0.1743, rmse:0.3726, r2:0.7867, pccs:0.887
property4: mae:0.1116, rmse:0.1776, r2:0.9645, pccs:0.9822
property5: mae:0.2429, rmse:1.6144, r2:0.0255, pccs:0.1688
property6: mae:0.5387, rmse:0.7189, r2:0.4485, pccs:0.6699
test
property1: mae:0.0823, rmse:0.1887, r2:0.6948, pccs:0.841
property2: mae:0.0746, rmse:0.1177, r2:0.9834, pccs:0.9929
property3: mae:0.1194, rmse:0.2959, r2:0.846, pccs:0.9201
property4: mae:0.0704, rmse:0.0981, r2:0.9896, pccs:0.9951
property5: mae:0.1162, rmse:0.5328, r2:0.0297, p

## p12

In [11]:
train = pd.read_csv("p12_train_scaling_all.csv")
train['istrain'] = 1
test = pd.read_csv("p12_test_scaling_all.csv")
test['istrain'] = 0
df = pd.concat([train, test]).reset_index(drop=True)

X_name = [f'element {i}' for i in range(1,13)]
Y_name = [f'property {i}' for i in range(1,7)]

poolx = df[X_name].copy(deep=True)
pooly = df[Y_name].copy(deep=True)

X = poolx.iloc[df[df['istrain']==1].index, :]
Y = pooly.iloc[df[df['istrain']==1].index, :]

X_test = poolx.iloc[df[df['istrain']==0].index, :].reset_index(drop=True)
Y_test = pooly.iloc[df[df['istrain']==0].index, :].reset_index(drop=True)

pred = torch.zeros((X_test.shape[0], 6))

X.shape

(1372, 12)

In [12]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error



use_fea = X.columns.tolist()
cat_idxs = []
cat_dims = []# qcut bins
cat_emb_dim = []
max_epochs = 1000

oof_train =  np.zeros(Y.shape)
oof_real_train = torch.zeros((X.shape[0], 6))
models = []

kf = KFold(n_splits=5, random_state=2022, shuffle=True)
for now_fold, (trn_idx, val_idx) in enumerate(kf.split(X, Y)):
    print(f"Fold: {now_fold+1}")
    X_train, X_val = X.iloc[trn_idx].values, X.iloc[val_idx].values
    y_train, y_val = Y.iloc[trn_idx].values, Y.iloc[val_idx].values
    
    clf = TabNetRegressor(seed=2022)
    
    clf.fit(
        X_train=X_train, y_train=y_train, 
        eval_set = [(X_train, y_train), (X_val, y_val)],
        eval_name=['train', 'valid'],
        eval_metric=['mae', 'rmse', 'mse', ], 
        max_epochs=max_epochs, 
        patience=50,
        batch_size=128, virtual_batch_size=32,
        num_workers=0,
        drop_last=False)
    

    oof_train[val_idx] = clf.predict(X_val)
    oof_real_train[trn_idx] += clf.predict(X_train) / 4
    
    pred += clf.predict(X_test.values) / 5
    
    models.append(clf)
    valid_mse = mean_squared_error(y_pred=oof_train[val_idx], y_true=y_val)


    print(f"FINAL VALID SCORE : {valid_mse}")

Fold: 1
Device used : cuda
epoch 0  | loss: 1.29727 | train_mae: 0.68444 | train_rmse: 0.92082 | train_mse: 0.84791 | valid_mae: 0.69056 | valid_rmse: 1.06414 | valid_mse: 1.13239 |  0:00:00s
epoch 1  | loss: 0.77333 | train_mae: 0.56191 | train_rmse: 0.79903 | train_mse: 0.63845 | valid_mae: 0.57007 | valid_rmse: 0.97165 | valid_mse: 0.94411 |  0:00:00s
epoch 2  | loss: 0.60836 | train_mae: 0.45616 | train_rmse: 0.71765 | train_mse: 0.51502 | valid_mae: 0.45738 | valid_rmse: 0.88996 | valid_mse: 0.79203 |  0:00:01s
epoch 3  | loss: 0.51738 | train_mae: 0.368   | train_rmse: 0.65593 | train_mse: 0.43024 | valid_mae: 0.36309 | valid_rmse: 0.8391  | valid_mse: 0.70408 |  0:00:01s
epoch 4  | loss: 0.44165 | train_mae: 0.32969 | train_rmse: 0.63524 | train_mse: 0.40353 | valid_mae: 0.33147 | valid_rmse: 0.83397 | valid_mse: 0.6955  |  0:00:02s
epoch 5  | loss: 0.4135  | train_mae: 0.32909 | train_rmse: 0.62796 | train_mse: 0.39433 | valid_mae: 0.33303 | valid_rmse: 0.81709 | valid_mse: 0.6

In [13]:
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, r2_score
print("train")

for i in range(Y.shape[1]):
    mae = mean_absolute_error(Y.iloc[:,i], oof_real_train[:,i])
    r2 = r2_score(Y.iloc[:,i], oof_real_train[:,i])
    rmse = np.sqrt(((Y.iloc[:, i] - np.array(oof_real_train[:,i])) ** 2).mean())
    pccs = pearsonr(Y.iloc[:, i], oof_real_train[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

print("valid")

for i in range(Y.shape[1]):
    mae = mean_absolute_error(Y.iloc[:,i], oof_train[:,i])
    r2 = r2_score(Y.iloc[:,i], oof_train[:,i])
    rmse = np.sqrt(((Y.iloc[:, i] - np.array(oof_train[:,i])) ** 2).mean())
    pccs = pearsonr(Y.iloc[:, i], oof_train[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

print("test")
for i in range(Y_test.shape[1]):
    mae = mean_absolute_error(Y_test.iloc[:,i], pred[:,i])
    r2 = r2_score(Y_test.iloc[:,i], pred[:,i])
    rmse = np.sqrt(((Y_test.iloc[:, i] - np.array(pred[:,i])) ** 2).mean())
    pccs = pearsonr(Y_test.iloc[:, i], pred[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

train
property1: mae:0.1261, rmse:0.2284, r2:0.9501, pccs:0.9765
property2: mae:0.0592, rmse:0.0858, r2:0.9925, pccs:0.9962
property3: mae:0.0633, rmse:0.1138, r2:0.9869, pccs:0.9934
property4: mae:0.0538, rmse:0.0779, r2:0.9939, pccs:0.9969
property5: mae:0.1413, rmse:1.0585, r2:0.1779, pccs:0.473
property6: mae:0.4802, rmse:0.6387, r2:0.5909, pccs:0.774
valid
property1: mae:0.1617, rmse:0.3145, r2:0.9054, pccs:0.9517
property2: mae:0.0771, rmse:0.1075, r2:0.9882, pccs:0.9941
property3: mae:0.0826, rmse:0.131, r2:0.9826, pccs:0.9913
property4: mae:0.0743, rmse:0.1017, r2:0.9895, pccs:0.9948
property5: mae:0.1756, rmse:1.1662, r2:0.002, pccs:0.0654
property6: mae:0.5204, rmse:0.691, r2:0.5213, pccs:0.7226
test
property1: mae:0.1167, rmse:0.2261, r2:0.942, pccs:0.972
property2: mae:0.0534, rmse:0.075, r2:0.9946, pccs:0.9973
property3: mae:0.0555, rmse:0.0963, r2:0.991, pccs:0.9955
property4: mae:0.0496, rmse:0.0725, r2:0.9949, pccs:0.9974
property5: mae:0.0848, rmse:0.3983, r2:-0.0347, 

## p13

In [2]:
train = pd.read_csv("p13_train_fe_hs.csv")
train['istrain'] = 1
test = pd.read_csv("p13_test_fe_hs.csv")
test['istrain'] = 0
df = pd.concat([train, test]).reset_index(drop=True)

X_name = df.columns.tolist()[:-7]
Y_name = [f'property {i}' for i in range(1,7)]

poolx = df[X_name].copy(deep=True)
pooly = df[Y_name].copy(deep=True)

X = poolx.iloc[df[df['istrain']==1].index, :]
Y = pooly.iloc[df[df['istrain']==1].index, :]

X_test = poolx.iloc[df[df['istrain']==0].index, :].reset_index(drop=True)
Y_test = pooly.iloc[df[df['istrain']==0].index, :].reset_index(drop=True)

pred = torch.zeros((X_test.shape[0], 6))
print(X.shape, X_test.shape)

(1372, 79) (588, 79)


In [3]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error



use_fea = X.columns.tolist()
cat_idxs = []
cat_dims = []# qcut bins
cat_emb_dim = []
max_epochs = 1000

oof_train =  np.zeros(Y.shape)
oof_real_train = torch.zeros((X.shape[0], 6))
models = []

kf = KFold(n_splits=5, random_state=2022, shuffle=True)
for now_fold, (trn_idx, val_idx) in enumerate(kf.split(X, Y)):
    print(f"Fold: {now_fold+1}")
    X_train, X_val = X.iloc[trn_idx].values, X.iloc[val_idx].values
    y_train, y_val = Y.iloc[trn_idx].values, Y.iloc[val_idx].values
    
    clf = TabNetRegressor(seed=2022)
    
    clf.fit(
        X_train=X_train, y_train=y_train, 
        eval_set = [(X_train, y_train), (X_val, y_val)],
        eval_name=['train', 'valid'],
        eval_metric=['mae', 'rmse', 'mse', ], 
        max_epochs=max_epochs, 
        patience=50,
        batch_size=128, virtual_batch_size=32,
        num_workers=0,
        drop_last=False)
    

    oof_train[val_idx] = clf.predict(X_val)
    oof_real_train[trn_idx] += clf.predict(X_train) / 4
    
    pred += clf.predict(X_test.values) / 5
    
    models.append(clf)
    valid_mse = mean_squared_error(y_pred=oof_train[val_idx], y_true=y_val)


    print(f"FINAL VALID SCORE : {valid_mse}")

Fold: 1
Device used : cuda
epoch 0  | loss: 1.86275 | train_mae: 0.63014 | train_rmse: 1.02256 | train_mse: 1.04563 | valid_mae: 0.68758 | valid_rmse: 1.93928 | valid_mse: 3.76079 |  0:00:01s
epoch 1  | loss: 1.16957 | train_mae: 0.62517 | train_rmse: 1.01615 | train_mse: 1.03257 | valid_mae: 0.68106 | valid_rmse: 1.93587 | valid_mse: 3.74761 |  0:00:02s
epoch 2  | loss: 1.02496 | train_mae: 0.61001 | train_rmse: 1.00264 | train_mse: 1.00528 | valid_mae: 0.66091 | valid_rmse: 1.9274  | valid_mse: 3.71487 |  0:00:02s
epoch 3  | loss: 0.91496 | train_mae: 0.54678 | train_rmse: 0.96337 | train_mse: 0.92807 | valid_mae: 0.58439 | valid_rmse: 1.90166 | valid_mse: 3.61632 |  0:00:03s
epoch 4  | loss: 0.81447 | train_mae: 0.45566 | train_rmse: 0.92504 | train_mse: 0.85569 | valid_mae: 0.48502 | valid_rmse: 1.88206 | valid_mse: 3.54213 |  0:00:03s
epoch 5  | loss: 0.68718 | train_mae: 0.44615 | train_rmse: 0.90703 | train_mse: 0.8227  | valid_mae: 0.47857 | valid_rmse: 1.87484 | valid_mse: 3.5

In [4]:
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, r2_score
print("train")

for i in range(Y.shape[1]):
    mae = mean_absolute_error(Y.iloc[:,i], oof_real_train[:,i])
    r2 = r2_score(Y.iloc[:,i], oof_real_train[:,i])
    rmse = np.sqrt(((Y.iloc[:, i] - np.array(oof_real_train[:,i])) ** 2).mean())
    pccs = pearsonr(Y.iloc[:, i], oof_real_train[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

print("valid")

for i in range(Y.shape[1]):
    mae = mean_absolute_error(Y.iloc[:,i], oof_train[:,i])
    r2 = r2_score(Y.iloc[:,i], oof_train[:,i])
    rmse = np.sqrt(((Y.iloc[:, i] - np.array(oof_train[:,i])) ** 2).mean())
    pccs = pearsonr(Y.iloc[:, i], oof_train[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

print("test")
for i in range(Y_test.shape[1]):
    mae = mean_absolute_error(Y_test.iloc[:,i], pred[:,i])
    r2 = r2_score(Y_test.iloc[:,i], pred[:,i])
    rmse = np.sqrt(((Y_test.iloc[:, i] - np.array(pred[:,i])) ** 2).mean())
    pccs = pearsonr(Y_test.iloc[:, i], pred[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

train
property1: mae:0.1429, rmse:0.6869, r2:0.8495, pccs:0.9413
property2: mae:0.0733, rmse:0.1077, r2:0.9851, pccs:0.9931
property3: mae:0.115, rmse:0.3089, r2:0.8534, pccs:0.9355
property4: mae:0.0777, rmse:0.098, r2:0.9892, pccs:0.9954
property5: mae:0.2044, rmse:1.5902, r2:0.0546, pccs:0.2363
property6: mae:0.5008, rmse:0.6652, r2:0.5278, pccs:0.7359
valid
property1: mae:0.1588, rmse:0.613, r2:0.8801, pccs:0.9394
property2: mae:0.0941, rmse:0.1422, r2:0.9741, pccs:0.9871
property3: mae:0.1687, rmse:0.3932, r2:0.7624, pccs:0.8751
property4: mae:0.0916, rmse:0.1271, r2:0.9818, pccs:0.9914
property5: mae:0.2311, rmse:1.617, r2:0.0224, pccs:0.1582
property6: mae:0.5479, rmse:0.7198, r2:0.4471, pccs:0.6687
test
property1: mae:0.1108, rmse:0.2576, r2:0.4309, pccs:0.6775
property2: mae:0.0744, rmse:0.1093, r2:0.9857, pccs:0.9939
property3: mae:0.0991, rmse:0.2638, r2:0.8775, pccs:0.9438
property4: mae:0.0717, rmse:0.0893, r2:0.9913, pccs:0.9963
property5: mae:0.1117, rmse:0.5231, r2:0.06

## p14

In [5]:
train = pd.read_csv("p14_train_fe_hsp.csv")
train['istrain'] = 1
test = pd.read_csv("p14_test_fe_hsp.csv")
test['istrain'] = 0
df = pd.concat([train, test]).reset_index(drop=True)

X_name = df.columns.tolist()[:-7]
Y_name = [f'property {i}' for i in range(1,7)]

poolx = df[X_name].copy(deep=True)
pooly = df[Y_name].copy(deep=True)

X = poolx.iloc[df[df['istrain']==1].index, :]
Y = pooly.iloc[df[df['istrain']==1].index, :]

X_test = poolx.iloc[df[df['istrain']==0].index, :].reset_index(drop=True)
Y_test = pooly.iloc[df[df['istrain']==0].index, :].reset_index(drop=True)

pred = torch.zeros((X_test.shape[0], 6))
print(X.shape, X_test.shape)

(1372, 145) (588, 145)


In [6]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error



use_fea = X.columns.tolist()
cat_idxs = []
cat_dims = []# qcut bins
cat_emb_dim = []
max_epochs = 1000

oof_train =  np.zeros(Y.shape)
oof_real_train = torch.zeros((X.shape[0], 6))
models = []

kf = KFold(n_splits=5, random_state=2022, shuffle=True)
for now_fold, (trn_idx, val_idx) in enumerate(kf.split(X, Y)):
    print(f"Fold: {now_fold+1}")
    X_train, X_val = X.iloc[trn_idx].values, X.iloc[val_idx].values
    y_train, y_val = Y.iloc[trn_idx].values, Y.iloc[val_idx].values
    
    clf = TabNetRegressor(seed=2022)
    
    clf.fit(
        X_train=X_train, y_train=y_train, 
        eval_set = [(X_train, y_train), (X_val, y_val)],
        eval_name=['train', 'valid'],
        eval_metric=['mae', 'rmse', 'mse', ], 
        max_epochs=max_epochs, 
        patience=50,
        batch_size=128, virtual_batch_size=32,
        num_workers=0,
        drop_last=False)
    

    oof_train[val_idx] = clf.predict(X_val)
    oof_real_train[trn_idx] += clf.predict(X_train) / 4
    
    pred += clf.predict(X_test.values) / 5
    
    models.append(clf)
    valid_mse = mean_squared_error(y_pred=oof_train[val_idx], y_true=y_val)


    print(f"FINAL VALID SCORE : {valid_mse}")

Fold: 1
Device used : cuda
epoch 0  | loss: 2.14289 | train_mae: 0.63546 | train_rmse: 1.02366 | train_mse: 1.04788 | valid_mae: 0.69311 | valid_rmse: 1.94164 | valid_mse: 3.76996 |  0:00:00s
epoch 1  | loss: 1.25212 | train_mae: 0.62787 | train_rmse: 1.013   | train_mse: 1.02616 | valid_mae: 0.68278 | valid_rmse: 1.92992 | valid_mse: 3.7246  |  0:00:00s
epoch 2  | loss: 1.10156 | train_mae: 0.63149 | train_rmse: 1.00157 | train_mse: 1.00314 | valid_mae: 0.68218 | valid_rmse: 1.91866 | valid_mse: 3.68125 |  0:00:01s
epoch 3  | loss: 1.01808 | train_mae: 0.62042 | train_rmse: 0.98989 | train_mse: 0.97988 | valid_mae: 0.6624  | valid_rmse: 1.90821 | valid_mse: 3.64125 |  0:00:01s
epoch 4  | loss: 0.92889 | train_mae: 0.59124 | train_rmse: 0.98653 | train_mse: 0.97323 | valid_mae: 0.62648 | valid_rmse: 1.90697 | valid_mse: 3.63654 |  0:00:02s
epoch 5  | loss: 0.86634 | train_mae: 0.58439 | train_rmse: 0.99039 | train_mse: 0.98087 | valid_mae: 0.61691 | valid_rmse: 1.90686 | valid_mse: 3.6

In [7]:
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, r2_score
print("train")

for i in range(Y.shape[1]):
    mae = mean_absolute_error(Y.iloc[:,i], oof_real_train[:,i])
    r2 = r2_score(Y.iloc[:,i], oof_real_train[:,i])
    rmse = np.sqrt(((Y.iloc[:, i] - np.array(oof_real_train[:,i])) ** 2).mean())
    pccs = pearsonr(Y.iloc[:, i], oof_real_train[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

print("valid")

for i in range(Y.shape[1]):
    mae = mean_absolute_error(Y.iloc[:,i], oof_train[:,i])
    r2 = r2_score(Y.iloc[:,i], oof_train[:,i])
    rmse = np.sqrt(((Y.iloc[:, i] - np.array(oof_train[:,i])) ** 2).mean())
    pccs = pearsonr(Y.iloc[:, i], oof_train[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

print("test")
for i in range(Y_test.shape[1]):
    mae = mean_absolute_error(Y_test.iloc[:,i], pred[:,i])
    r2 = r2_score(Y_test.iloc[:,i], pred[:,i])
    rmse = np.sqrt(((Y_test.iloc[:, i] - np.array(pred[:,i])) ** 2).mean())
    pccs = pearsonr(Y_test.iloc[:, i], pred[:,i])[0]
    mae, r2, rmse, pccs = np.around([mae, r2, rmse, pccs], decimals=4)
    print(f'property{i+1}: mae:{mae}, rmse:{rmse}, r2:{r2}, pccs:{pccs}')

train
property1: mae:0.1633, rmse:1.7007, r2:0.0771, pccs:0.9758
property2: mae:0.0747, rmse:0.1204, r2:0.9814, pccs:0.992
property3: mae:0.122, rmse:0.3365, r2:0.8259, pccs:0.9255
property4: mae:0.0805, rmse:0.116, r2:0.9849, pccs:0.993
property5: mae:0.2014, rmse:1.5684, r2:0.0803, pccs:0.3145
property6: mae:0.5047, rmse:0.6581, r2:0.5379, pccs:0.7384
valid
property1: mae:0.1759, rmse:1.1226, r2:0.5979, pccs:0.8052
property2: mae:0.105, rmse:0.1823, r2:0.9574, pccs:0.9791
property3: mae:0.1719, rmse:0.4029, r2:0.7505, pccs:0.8721
property4: mae:0.1042, rmse:0.1774, r2:0.9646, pccs:0.9824
property5: mae:0.2462, rmse:1.623, r2:0.0151, pccs:0.1355
property6: mae:0.5437, rmse:0.7197, r2:0.4473, pccs:0.6688
test
property1: mae:0.0859, rmse:0.217, r2:0.5962, pccs:0.7728
property2: mae:0.0709, rmse:0.112, r2:0.985, pccs:0.9948
property3: mae:0.1028, rmse:0.2762, r2:0.8658, pccs:0.9423
property4: mae:0.0711, rmse:0.0962, r2:0.9899, pccs:0.9956
property5: mae:0.1003, rmse:0.479, r2:0.2159, pc